In [176]:
import pandas as pd
from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import os
import sliding_window
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import importlib
importlib.reload(sliding_window)

<module 'sliding_window' from 'C:\\Users\\suhwanlee\\Desktop\\project\\streaming_anomaly_detect\\sliding_window.py'>

In [185]:
dataset = stream.iter_csv(
            './bpic17_length28.csv',
            )

totallength = len(list(dataset))

In [186]:
dataset = stream.iter_csv(
            './bpic17_length28.csv',
#             drop=['Complete Timestamp'],
            )
enctype = 'Index-base'

In [187]:
key_pair = {
# 'Case ID':'caseid',
# 'Activity':'activity',
# 'Resource':'resource',
# 'Complete Timestamp':'ts',
}
catatars= ['activity','resource']

case_dict ={}
training_models ={}

casecount = 0
rowcounter = 0
resultdict ={}
acc_dict ={}

prediction_result = {}
graceperiod_finish=0
finishedcases = set()

In [188]:
# Sliding window for training setting
window_size = 20
training_window = sliding_window.training_window(window_size)


In [189]:
def display_progress(row_counting, total_length, interval=500):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount))

In [190]:
def training_stage(window, training_models):
    '''
    Manage training stage of streaming anomaly detection
    ----------
    Parameters
    window: class training_window
        Sliding window with training data
    training_models: dict
        Trained detector by prefix stored in. Default is randomforest
    ----------
    Return
    training_models
    '''
    pw_window = window.prefix_wise_window()
    for x in pw_window:
        clf  = DecisionTreeClassifier(max_depth=10)
        clf.fit(pw_window[x][0],pw_window[x][1])
        if 'detector_%s'%(x) not in training_models:
            training_models['detector_%s'%(x)] =[0,0]
        training_models['detector_%s'%(x)][0] += 1
        training_models['detector_%s'%(x)][1] = clf
    return training_models

In [191]:
for x,y in dataset:
    display_progress(rowcounter, totallength)
    rowcounter +=1
    
    utils.dictkey_chg(x, key_pair)
    # Event stream change dictionary keys
    x['ts'] = x['ts'][:-4]
    
    # Check label possible
    
    # Initialize case by prefix length
    caseid = x['caseid']
    x.pop('caseid')
    
    case_bin = prefix_bin(caseid, x)
    
    if caseid not in list(case_dict.keys()):
        case_dict[caseid] = []
        case_bin.set_prefix_length(1)
        
    elif caseid in finishedcases:
        continue
    
    else:
        case_bin.set_prefix_length(len(case_dict[caseid])+1)
        case_bin.set_prev_enc(case_dict[caseid][-1])
    
    # Encode event and cases and add to DB
    ts = case_bin.event['ts']
    case_bin.update_encoded(catattrs=catatars,enctype=enctype)
    
    # Set current activity as outcome of previous event
    if case_bin.prefix_length != 1:
        case_bin.prev_enc.update_truelabel(x['activity'])

    # Update training window and finish the case
    if x['activity'] == 'End':
        training_window.update_window({caseid: case_dict[caseid]})
        training_models = training_stage(training_window, training_models)
        case_dict.pop(caseid)
        casecount +=1
    else:
        case_dict[caseid].append(case_bin)
    
    # Predict probability of next activities
    # If 
    
    for x in case_dict:
        print(case_dict[x][-1].encoded)
#     if len(training_window.getAllitems()) !=0:
#         prefix_wise_window = training_window.prefix_wise_window()
        
#         if 'window_%s'%(case_bin.prefix_length) in list(prefix_wise_window.keys()) and 'detector_%s'%(case_bin.prefix_length) in training_models.keys():
#             feature_matrix = {'window_%s'%(case_bin.prefix_length): prefix_wise_window['window_%s'%(case_bin.prefix_length)][0].columns.values}
#             current_event = utils.readjustment_training(case_bin.encoded, feature_matrix)
#             prediction = training_models['window_%s'%(case_bin.prefix_length)][1].predict(current_event)
#         else:
#             prediction = 'Not Available'
#     else:
#         prediction = 'Not Available'
#     case_dict[caseid][-1].update_prediction((training_models['detector_%s'%(case_bin.prefix_length)][0], (prediction,ts)))


    

0.0 % Case finished: 0
6.22 % Case finished: 8


KeyboardInterrupt: 

In [194]:

# print(case_dict)

{'prefix_length': 14, 'caseid': 'Application_1141114991', 'event': {'activity': 'W_Call after offers', 'resource': 'User_15', 'ts': datetime.datetime(2016, 1, 8, 10, 21, 49)}, 'enctype': 'Index-base', 'start_ts': datetime.datetime(2016, 1, 4, 19, 47, 14), 'prev_enc': <encoding.prefix_bin object at 0x0000018D8E790B80>, 'encoded': {'duration_1': 0, 'cumduration_1': 0, 'activity_1 A_Create Application': 1, 'resource_1 User_43': 1, 'duration_2': 0.0, 'cumduration_2': 0.0, 'activity_2 W_Complete application': 1, 'resource_2 User_43': 1, 'duration_3': 0.0, 'cumduration_3': 0.0, 'activity_3 W_Complete application': 1, 'resource_3 User_43': 1, 'duration_4': 0.0, 'cumduration_4': 0.0, 'activity_4 A_Concept': 1, 'resource_4 User_43': 1, 'duration_5': 887.0, 'cumduration_5': 887.0, 'activity_5 A_Accepted': 1, 'resource_5 User_43': 1, 'duration_6': 116.0, 'cumduration_6': 1003.0, 'activity_6 O_Create Offer': 1, 'resource_6 User_43': 1, 'duration_7': 2.0, 'cumduration_7': 1005.0, 'activity_7 O_Crea

In [85]:
importlib.reload(sliding_window)
pw_window = training_window.prefix_wise_window()

27


In [103]:
print(pw_window['window_2'][0].columns.values)

['duration_1' 'cumduration_1' 'activity_1 A_Create Application'
 'resource_1 User_5' 'duration_2' 'cumduration_2'
 'activity_2 W_Complete application' 'resource_2 User_5'
 'resource_1 User_1' 'activity_2 A_Submitted' 'resource_2 User_1'
 'resource_1 User_47' 'resource_2 User_47' 'resource_1 User_3'
 'activity_2 A_Concept' 'resource_2 User_3' 'resource_1 User_28'
 'resource_2 User_28' 'resource_1 User_23' 'resource_2 User_23'
 'resource_1 User_16' 'resource_2 User_16' 'resource_1 User_76'
 'resource_2 User_76' 'resource_1 User_45' 'resource_2 User_45'
 'resource_1 User_37' 'resource_2 User_37' 'resource_1 User_52'
 'resource_2 User_52' 'resource_1 User_4' 'resource_2 User_4'
 'resource_1 User_10' 'resource_2 User_10' 'resource_1 User_51'
 'resource_2 User_51']


In [99]:
training_models = training_stage(training_window, training_models)

In [68]:
print(clf.classes_)
print(clf.predict_proba(x_train.loc[19:]))
print(clf.predict(x_train.loc[19:]))

['A_Accepted' 'O_Create Offer' 'O_Created' 'W_Complete application']
[[0.06 0.   0.91 0.03]]
['O_Created']


In [62]:
print(clf.score(x_train,y_train))

1.0
